# Functions for the evolutionary models

This file contains all functions necessary for fitting and plotting the evolutionary models. 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import scipy.integrate as it
from scipy.stats import kde
import pandas as pd
from scipy.special import gamma
from IPython.display import clear_output
import scipy.optimize as opt
from scipy.stats import norm
import os
import datetime


import matplotlib
matplotlib.rc_file_defaults()
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'Lato'

Density of clone sizes and frequencies (equations 1 and 2, supplementary material)

$$ \rho(n, t, \{a_i\}) dn = n^{\theta - 1} \frac{e^{-n/\tilde{n}(t)}}{\Gamma(\theta)\tilde{n}(t)^{\theta}} dn  \qquad \qquad (1)$$

$$\textrm{where}  \quad \theta = N\tau\mu \quad \textrm{and} \quad \tilde{n}(t) = \frac{\exp(st) - 1}{s\tau}$$

$$ \rho(f, t, \{a_i\}) df =  \frac{(2N)^\theta}{f^{1-\theta}(1-2f)^{1+\theta}} \exp\left(\frac{-2Nf}{\tilde{n}(t)(1-2f)}\right) \frac{1}{\Gamma(\theta)\tilde{n}(t)^\theta} df   \qquad \qquad (2) $$



In [6]:
# analytic density functions - generic and non-approximate
# with n as argument
def rho_of_n(n, mu, s, age, N):

    n_tilde = (np.exp(s*age)-1)/s
    theta = mu*N
    return 1/(n**(1-theta)) * (np.exp(-n/n_tilde)) / (gamma(theta)*(n_tilde**theta))

# with f as argument (with option to adjust by n2 (background), default n2=0
def rho_of_f(f, mu, s, age, N, n2=0):

    n_tilde = (np.exp(s*age)-1)/s
    theta = mu*N

    # broken into parts for easy reading (see expression above)
    coeff = (2*(N+n2))**theta / (1-2*f)**(theta+1)
    main_freq_dependence = 1/(f**(1-theta))
    in_exponential = (-2*(N+n2)*f)/(n_tilde*(1-2*f))
    adjustment = 1/(gamma(theta)*n_tilde**theta)

    return coeff*main_freq_dependence*np.exp(in_exponential)*adjustment

Adjusting for sequencing depth (eq 3, supplementary material)

$$ \rho_d(r|D, t, \{a_i\}) = \int_0^{0.5} P(r|D, f) \ \rho(f, t) df \qquad \qquad (3) $$

$$ \textrm{Where} \quad P(r|D, f) = \binom{D}{r} f^r (1-f)^{D-r}$$


In [7]:
def sampling_integrand(f, mu, s, age, N, reads, depth):

    f_density = rho_of_f(f, mu, s, age, N)

    return f_density * scipy.stats.binom.pmf(reads, depth, f)


def rho_with_sampling(r, depth, mu, s, age, N):

    f1_integration_limits = [0, 0.5]
    I = it.quad(sampling_integrand, *f1_integration_limits, args=(mu, s, age, N, r, depth))

    return I[0]

Calculate cumulative square distance using the log-cumulative density

In [8]:
def cum_square_logdistance(optimisation_parameters, variants, N, number_of_individuals):

    mu, s = optimisation_parameters
    D = int(variants.depth.mean())
    x = np.arange(D+1)/D
    n_variants = len(variants)

    # calculate the cumulative curve
    density = np.zeros(D+1)
    for row in biobank_bins_quantiles.index:
        age = biobank_bins_quantiles['Age.when.attended.assessment.centre_v0'][row]
        age_count = biobank_bins_quantiles['age_count'][row]
        for r in range(D + 1):
            increment = rho_with_sampling(r, D, mu, s, age, N) * age_count
            density[r] += increment / number_of_individuals

    cumul_density = np.cumsum(density[::-1])[::-1]

    variants_ordered = variants.sort_values('VAF', ascending=False)
    y_variants = (np.arange(n_variants)+1)/number_of_individuals

    density_list = []
    for x in variants_ordered.VAF:
        r_exact = x*D
        r_decimal, r_lower = np.modf(r_exact)
        r_lower = int(r_lower)
        r_higher = int(np.ceil(r_exact))
        density_higher = cumul_density[r_higher]
        density_lower = cumul_density[r_lower]
        density_list.append(density_higher**r_decimal * density_lower**(1-r_decimal))

    log_actual = np.log(y_variants)
    log_theoretical = np.log(np.array(density_list))

    square_distance = sum((log_actual-log_theoretical)**2)
    return square_distance

Calculate single fitness model density based on the age distribution of the samples

In [ ]:
def single_fitness_predictions_calc(mu, s, N, variant_quantiles):

    D = int(variant_quantiles.depth.mean())
    number_of_individuals = biobank_ages_bins.age_count.sum()

    sampled_quantile_density = {}
    cumul_quantile_density = {}
    sampled_overall_density = np.zeros(D+1)

    for qt in range(number_of_quantiles):
        quantile_age_bins = biobank_bins_quantiles[biobank_bins_quantiles.quantile_labels == qt]
        n_in_quantile = quantile_age_bins.age_count.sum()

        # calculate sampled density
        sampled_quantile_density[qt] = np.zeros(D + 1)
        for row in quantile_age_bins.index:
            age = quantile_age_bins['Age.when.attended.assessment.centre_v0'][row]
            age_count = quantile_age_bins['age_count'][row]

            for r in range(D + 1):
                increment = rho_with_sampling(r, D, mu, s, age, N) * age_count
                sampled_quantile_density[qt][r] += increment / n_in_quantile
                sampled_overall_density[r] += increment / number_of_individuals

                #clear_output() this stuff actually seriously slows it down
                #print(age, age_count)
                #print(r, '/', D)

        cumul_quantile_density[qt] = np.cumsum(sampled_quantile_density[qt][::-1])[::-1]


    cumul_overall_density = np.cumsum(sampled_overall_density[::-1])[::-1]

    return cumul_overall_density, cumul_quantile_density
